In [1]:
from tpm.data_model import *
from tpm.util.io import read_geolife
from tpm.util.dist import haversine_distance
from tpm.preprocessing import time_duplication_filter
from tpm.preprocessing import speed_filter_abs
import numpy as np
import pandas as pd
import folium
from datetime import timedelta

In [2]:
trajs = read_geolife('/mnt/hdd1/christian/data/geotracking/Geolife Trajectories 1.3/Data/005/Trajectory')


In [3]:
preprocessed = list()
for traj in trajs:
    traj_new = time_duplication_filter(traj)
    traj_new = speed_filter_abs(traj_new, 300, in_kmh=True)
    preprocessed.append(traj_new)


In [4]:
def staypoints_geolife(traj):
    time_thresh = 30*60
    dist_thresh = 250

    staypoints = list()
    i, i_max = 0, len(traj)
    while i < i_max:
        j = i+1
        token = 0
        while j < i_max:
            dist = haversine_distance(traj[i], traj[j])
            if dist > dist_thresh:
                delta_time = traj[j].datetime - traj[i].datetime
                if delta_time.total_seconds() > time_thresh:
                    mean_point = np.mean([[p.lat, p.lon] for p in traj[i:j+1]], axis=0)
                    arrival_time = traj[i].datetime
                    leave_time = traj[j].datetime
                    staypoints.append([mean_point, arrival_time, leave_time, i, j])
                    i = j
                    token = 1
                break
            j = j+1
        if not token == 1:
            i = i+1

    
    return staypoints

In [5]:
len(trajs)

86

In [6]:
def make_df(trajs):
    data = list()
    for traj in trajs:
        fp = traj[0]
        sps = staypoints_geolife(traj)
        lp = traj[-1]               
        
        if len(sps) > 1:
            data.append([fp.lat, fp.lon, fp.datetime, sps[0][0][0], sps[0][0][1], sps[0][1]])
            for i in range(1, len(sps)-1):
                data.append([sps[i][0][0], sps[i][0][1], sps[i][1], sps[i+1][0][0], sps[i+1][0][1], sps[i+1][2]])
            data.append([sps[-1][0][0], sps[-1][0][1], sps[-1][2], lp.lat, lp.lon, lp.datetime])
        else:
            data.append([fp.lat, fp.lon, fp.datetime, lp.lat, lp.lon, lp.datetime])
        
        
    df = pd.DataFrame(data, columns=['start_lat','start_lon','start_date','end_lat','end_lon','end_date'])
    df = df.set_index(pd.DatetimeIndex(df['start_date'])).sort_index()
    return df

In [7]:
df = make_df(preprocessed)
df


,start_lat,start_lon,start_date,end_lat,end_lon,end_date
2008-10-24 13:12:30,40.004154,116.321335,2008-10-24 13:12:30,39.999413,116.326248,2008-10-24 13:16:15
2008-10-24 18:11:01,39.959038,116.360809,2008-10-24 18:11:01,39.957645,116.356674,2008-10-24 20:32:13
2008-10-24 18:56:10,39.957645,116.356674,2008-10-24 18:56:10,39.957253,116.356514,2008-10-24 21:43:12
2008-10-24 20:32:52,39.957253,116.356514,2008-10-24 20:32:52,39.957027,116.356186,2008-10-24 23:08:52
2008-10-24 23:08:52,39.957027,116.356186,2008-10-24 23:08:52,40.010796,116.321587,2008-10-25 00:59:03
2008-10-25 13:17:08,39.999077,116.326668,2008-10-25 13:17:08,40.010548,116.321205,2008-10-25 13:50:08
2008-10-25 23:04:29,40.011021,116.321846,2008-10-25 23:04:29,40.011230,116.321518,2008-10-26 03:29:35
2008-10-26 18:44:26,39.996109,116.326599,2008-10-26 18:44:26,40.000584,116.326851,2008-10-27 00:03:11
2008-10-27 18:26:07,39.997608,116.326202,2008-10-27 18:26:07,39.939476,116.344353,2008-10-27 18:49:42
2008-10-27 19:43:47,39.906693,116.185890,2008-10-27 19:43:47,39.907146,116.208710,2008-10-27 22:30:22


In [8]:
from sklearn.cluster import dbscan
from sklearn.cluster import k_means
from sklearn.neighbors import DistanceMetric
from tpm.data_model import R
from collections import Counter

In [9]:
def haversine_distance(p1_lat, p1_lon, p2_lat, p2_lon):
    lat_rad1 = radians(p1_lat)
    lon_rad1 = radians(p1_lon)
    lat_rad2 = radians(p2_lat)
    lon_rad2 = radians(p2_lon)
    return 2*R * asin(sqrt(sin((lat_rad2-lat_rad1)/2)**2 + cos(lat_rad1)*cos(lat_rad2)*(sin((lon_rad2-lon_rad1)/2)**2)))

In [10]:
def cluster_into_spots(df, init_eps=150, levels=2, threshold=0.1):
    start_points = list()
    end_points = list()
    length = len(df)
    
    for i in range(length):
        start_points.append([df['start_lat'].iloc[i], df['start_lon'].iloc[i]])
        end_points.append([df['end_lat'].iloc[i], df['end_lon'].iloc[i]])
    
    points = np.radians(np.vstack([start_points, end_points]))
    
    haversine = DistanceMetric.get_metric('haversine')
    dist = haversine.pairwise(points)*R

    clusters = dbscan(dist, metric='precomputed', min_samples=1,eps=init_eps)[1]
    clusters = np.array(clusters, dtype=np.object)
    
    
    for _ in range(levels):
        init_eps = init_eps*0.3
        counts = dict(Counter(clusters))
        for key in counts:
            if counts[key] > threshold*length:
                idxs = np.where(clusters == key)[0]
                dist = haversine.pairwise(points[idxs])*R
                inner_clusters = dbscan(dist, metric='precomputed', min_samples=1,eps=init_eps)[1]
                for i, idx in enumerate(idxs):
                    
                    clusters[idx] = "{}_{}".format(clusters[idx], inner_clusters[i])
    
    start_clusters = list()
    end_clusters = list()
    for i, cluster in enumerate(clusters):
        if i < length:
            start_clusters.append(clusters[i])
        else:
            end_clusters.append(clusters[i%length+length])
    
    print(len(dict(Counter(clusters)).keys()))
    
    df['start_cluster'] = start_clusters
    df['end_cluster'] = end_clusters
    return df

In [11]:
def cluster_into_spots(df, init_eps=150, levels=2, threshold=0.1):
    start_points = list()
    end_points = list()
    length = len(df)
    
    for i in range(length):
        start_points.append([df['start_lat'].iloc[i], df['start_lon'].iloc[i]])
        end_points.append([df['end_lat'].iloc[i], df['end_lon'].iloc[i]])
    
    points = np.radians(np.vstack([start_points, end_points]))
    
    haversine = DistanceMetric.get_metric('haversine')
    dist = haversine.pairwise(points)*R

    clusters = dbscan(dist, metric='precomputed', min_samples=1,eps=init_eps)[1]
    clusters = np.array(clusters, dtype=np.object)
    
    k_means()
    
    for _ in range(levels):
        init_eps = init_eps*0.3
        counts = dict(Counter(clusters))
        for key in counts:
            if counts[key] > threshold*length:
                idxs = np.where(clusters == key)[0]
                dist = haversine.pairwise(points[idxs])*R
                inner_clusters = dbscan(dist, metric='precomputed', min_samples=1,eps=init_eps)[1]
                for i, idx in enumerate(idxs):
                    
                    clusters[idx] = "{}_{}".format(clusters[idx], inner_clusters[i])
    
    start_clusters = list()
    end_clusters = list()
    for i, cluster in enumerate(clusters):
        if i < length:
            start_clusters.append(clusters[i])
        else:
            end_clusters.append(clusters[i%length+length])
    
    print(len(dict(Counter(clusters)).keys()))
    
    df['start_cluster'] = start_clusters
    df['end_cluster'] = end_clusters
    return df

In [12]:
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import cnames
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

In [13]:
def agglomerative_cluster_into_spots(df, min_cluster, max_cluster):
    start_points = list()
    end_points = list()
    length = len(df)

    for i in range(length):
        start_points.append([df['start_lat'].iloc[i], df['start_lon'].iloc[i]])
        end_points.append([df['end_lat'].iloc[i], df['end_lon'].iloc[i]])

    points = np.radians(np.vstack([start_points, end_points]))

    sil_scores = list()
    for i in range(min_cluster, max_cluster):
        ac = AgglomerativeClustering(n_clusters=i)
        pred = ac.fit_predict(points)
        sil_score = silhouette_score(points, pred)
        sil_scores.append(sil_score)

    
    n_cluster = np.argmax(sil_scores) + min_cluster
    ac = AgglomerativeClustering(n_clusters=n_cluster)
    clusters = ac.fit_predict(points)
    
    
    start_clusters = list()
    end_clusters = list()
    
    for i, cluster in enumerate(clusters):
        if i < length:
            start_clusters.append(clusters[i])
        else:
            end_clusters.append(clusters[i%length+length])

        

    df['start_cluster'] = start_clusters
    df['end_cluster'] = end_clusters
    return df

In [14]:
def kmeans_cluster_into_spots(df, min_cluster, max_cluster):
    start_points = list()
    end_points = list()
    length = len(df)

    for i in range(length):
        start_points.append([df['start_lat'].iloc[i], df['start_lon'].iloc[i]])
        end_points.append([df['end_lat'].iloc[i], df['end_lon'].iloc[i]])

    points = np.radians(np.vstack([start_points, end_points]))

    sil_scores = list()
    for i in range(min_cluster, max_cluster):
        ac = AgglomerativeClustering(n_clusters=i)
        pred = ac.fit_predict(points)
        sil_score = silhouette_score(points, pred)
        sil_scores.append(sil_score)

    
    n_cluster = np.argmax(sil_scores) + min_cluster
    ac = KMeans(n_clusters=n_cluster, n_jobs=-1)
    clusters = ac.fit_predict(points)
    
    
    start_clusters = list()
    end_clusters = list()
    
    for i, cluster in enumerate(clusters):
        if i < length:
            start_clusters.append(clusters[i])
        else:
            end_clusters.append(clusters[i%length+length])

        

    df['start_cluster'] = start_clusters
    df['end_cluster'] = end_clusters
    return df

In [15]:
def meanshift_cluster_into_spots(df):
    start_points = list()
    end_points = list()
    length = len(df)

    for i in range(length):
        start_points.append([df['start_lat'].iloc[i], df['start_lon'].iloc[i]])
        end_points.append([df['end_lat'].iloc[i], df['end_lon'].iloc[i]])

    points = np.radians(np.vstack([start_points, end_points]))


    ac = MeanShift(n_jobs=-1)
    print('start clustering')
    clusters = ac.fit_predict(points)
    print('done clustering')
    
    start_clusters = list()
    end_clusters = list()
    
    for i, cluster in enumerate(clusters):
        if i < length:
            start_clusters.append(clusters[i])
        else:
            end_clusters.append(clusters[i%length+length])

        

    df['start_cluster'] = start_clusters
    df['end_cluster'] = end_clusters
    return df

In [16]:
n_cluster = np.argmax(sil_scores) +100
ac = AgglomerativeClustering(n_clusters=n_cluster)
pred = ac.fit_predict(points)
points_deg = np.rad2deg(points)
visualize_cluster(points_deg, pred)

NameError: name 'sil_scores' is not defined

In [17]:
Counter(pred)

NameError: name 'pred' is not defined

In [16]:
def visualize_cluster(df):
    colors = [hexc for hexc in cnames.values()]
    map_lat, map_lon = df.iloc[0].start_lat, df.iloc[0].start_lon
    map_osm = folium.Map(location=[map_lat, map_lon])

    for i, row in df.iterrows():
        tup = (row['start_lat'], row['start_lon'])
        start_c = row['start_cluster']
        marker = folium.CircleMarker(tup, color=colors[start_c], fill_color=colors[start_c], radius=50, fill_opacity=1)
        map_osm.add_children(marker)
        
        tup = (row['end_lat'], row['end_lon'])
        end_c = row['end_cluster']
        marker = folium.CircleMarker(tup, color=colors[end_c], fill_color=colors[end_c], radius=50, fill_opacity=1)
        map_osm.add_children(marker)

    return map_osm

In [ ]:
df = cluster_into_spots(df, init_eps=300, levels=1, threshold=0.5)

counter = 0

for i in range(1,len(df)):
    ec = df['end_cluster'].iloc[i-1]
    sc = df['start_cluster'].iloc[i]
    if ec == sc:
        counter += 1
        
print(counter, len(df))

In [ ]:
df = agglomerative_cluster_into_spots(df,100,120)

counter = 0

for i in range(1,len(df)):
    ec = df['end_cluster'].iloc[i-1]
    sc = df['start_cluster'].iloc[i]
    if ec == sc:
        counter += 1
        
print(counter, len(df))

In [20]:
df = kmeans_cluster_into_spots(df,10,50)

counter = 0

for i in range(1,len(df)):
    ec = df['end_cluster'].iloc[i-1]
    sc = df['start_cluster'].iloc[i]
    if ec == sc:
        counter += 1
        
print(counter, len(df))

131 174


In [ ]:
df = meanshift_cluster_into_spots(df)

counter = 0

for i in range(1,len(df)):
    ec = df['end_cluster'].iloc[i-1]
    sc = df['start_cluster'].iloc[i]
    if ec == sc:
        counter += 1
        
print(counter, len(df))

In [21]:
visualize_cluster(df)

In [19]:
df

,start_lat,start_lon,start_date,end_lat,end_lon,end_date,start_cluster,end_cluster
2008-10-24 13:12:30,40.004154,116.321335,2008-10-24 13:12:30,39.999413,116.326248,2008-10-24 13:16:15,83,87
2008-10-24 18:11:01,39.959038,116.360809,2008-10-24 18:11:01,39.957645,116.356674,2008-10-24 20:32:13,21,91
2008-10-24 18:56:10,39.957645,116.356674,2008-10-24 18:56:10,39.957253,116.356514,2008-10-24 21:43:12,91,9
2008-10-24 20:32:52,39.957253,116.356514,2008-10-24 20:32:52,39.957027,116.356186,2008-10-24 23:08:52,9,107
2008-10-24 23:08:52,39.957027,116.356186,2008-10-24 23:08:52,40.010796,116.321587,2008-10-25 00:59:03,107,24
2008-10-25 13:17:08,39.999077,116.326668,2008-10-25 13:17:08,40.010548,116.321205,2008-10-25 13:50:08,113,75
2008-10-25 23:04:29,40.011021,116.321846,2008-10-25 23:04:29,40.011230,116.321518,2008-10-26 03:29:35,105,78
2008-10-26 18:44:26,39.996109,116.326599,2008-10-26 18:44:26,40.000584,116.326851,2008-10-27 00:03:11,40,6
2008-10-27 18:26:07,39.997608,116.326202,2008-10-27 18:26:07,39.939476,116.344353,2008-10-27 18:49:42,26,3
2008-10-27 19:43:47,39.906693,116.185890,2008-10-27 19:43:47,39.907146,116.208710,2008-10-27 22:30:22,4,16


In [ ]:
def visualize_rows(rows):
    map_lat, map_lon = rows.iloc[0].start_lat, rows.iloc[0].start_lon
    map_osm = folium.Map(location=[map_lat, map_lon])

    for i, row in rows.iterrows():
        tup = (row['start_lat'], row['start_lon'])
        marker = folium.Marker(tup, icon=folium.Icon(color='green'), popup='{} {}'.format(i, row['start_cluster']))
        map_osm.add_children(marker)
        tup = (row['end_lat'], row['end_lon'])
        marker = folium.Marker(tup, icon=folium.Icon(color='red'), popup='{} {}'.format(i, row['end_cluster']))
        map_osm.add_children(marker)

    return map_osm

In [ ]:
visualize_rows(df.iloc[:-1])

In [22]:
ts = df['end_date'].iloc[-1] - df['start_date'].iloc[0]
ts.days/7

20.857142857142858

In [23]:
duration = timedelta(days=7*15)
train_start = df['start_date'].iloc[0]
train_end = train_start + duration
train = df[train_start:train_end]
test = df[train_end:]

In [25]:
dates = list()
for i in test.index:
    if len(dates) > 1:
        if dates[-1] == (i.month, i.day):
            continue
    dates.append((i.month, i.day))

In [26]:
from masterthesis.models import BayesWeekdayEstimator
from masterthesis.models import FrequentistEstimator
import operator

In [27]:
bwe = FrequentistEstimator()

bwe = bwe.fit(train)

counter_no_prob = 0

for date in dates:
    month, day = date
    x = pd.DataFrame(data=[[49.475752, 8.482531]],index=pd.DatetimeIndex([pd.Timestamp("2008-{}-{} 19:45:21".format(month,day))]), columns=['lat', 'lon'])
    print(x)
    pred = bwe.predict_proba(x)
    
    sorted_pred = sorted(pred.items(), key=operator.itemgetter(1), reverse=True)
    
    for i, row in df.loc[(df.index.month==month) & (df.index.day==day)].iterrows():
        print((row['start_cluster'], row['end_cluster']))
        if (row['start_cluster'], row['end_cluster']) in pred.keys():
            print(pred[(row['start_cluster'], row['end_cluster'])])
            for i, s_pred in enumerate(sorted_pred):
                if s_pred[0] == (row['start_cluster'], row['end_cluster']):
                    print('Ranked:', i+1, 'of total', len(pred), 'predictions')
        else:
            print("no prob")
            counter_no_prob += 1
            
    
print('Number of no prob:', counter_no_prob)
print('Total predictions:', len(test))

                           lat       lon
2008-02-08 19:45:21  49.475752  8.482531
(2, 2)
0.006493506493506494
Ranked: 25 of total 26 predictions
                           lat       lon
2008-02-22 19:45:21  49.475752  8.482531
(0, 0)
0.14935064935064934
Ranked: 4 of total 26 predictions
(0, 7)
0.2077922077922078
Ranked: 1 of total 26 predictions
(7, 7)
0.2012987012987013
Ranked: 3 of total 26 predictions
(7, 0)
0.2077922077922078
Ranked: 2 of total 26 predictions
                           lat       lon
2008-02-23 19:45:21  49.475752  8.482531
(0, 0)
0.14935064935064934
Ranked: 4 of total 26 predictions
                           lat       lon
2008-02-24 19:45:21  49.475752  8.482531
(0, 0)
0.14935064935064934
Ranked: 4 of total 26 predictions
                           lat       lon
2008-02-25 19:45:21  49.475752  8.482531
(0, 0)
0.14935064935064934
Ranked: 4 of total 26 predictions
(7, 0)
0.2077922077922078
Ranked: 2 of total 26 predictions
                           lat       lon
20

In [28]:
from masterthesis.preprocessing import DenseDepartureTimes
from masterthesis.models import BayesDepartureTimeEstimator
from sklearn.base import BaseEstimator
from sklearn.cluster import dbscan

In [29]:
class BayesDepartureTimeEstimator(BaseEstimator):
    def fit(self, X, y=None):
        self.data_ = X
        return self

    def partial_fit(self, X):
        # stack data to present data
        pass

    def predict_proba(self, x):
        length = len(self.data_)
        start_end = [(start, end) for start, end in zip(self.data_['start_cluster'], self.data_['end_cluster'])]
        priors = {str(k): v / length for k, v in Counter(start_end).items()}

        dayofweek = x.index.dayofweek
        
        p_ba = [row['start_time_cluster'] for index, row in self.data_.iterrows() if
                index.dayofweek == dayofweek]
        p_ba = {k: v / len(p_ba) for k, v in Counter(p_ba).items()}
        
        res = {key: priors[key.split('_')[0]] * p_ba[key] / (1 / 7) for key in p_ba}

        return res
    
        
    def resolve_start_time_cluster(self, stc):
        stc_df = self.data_[self.data_['start_time_cluster'] == stc]
        
        return min(stc_df.index.time), max(stc_df.index.time)

In [30]:
from datetime import datetime
from datetime import date
from datetime import time

In [31]:
def _time_to_degree(time):
    return ((time.hour + (time.minute + (time.second / 60)) / 60) / 24) * 360

def _time_distance(t1, t2):
    circumference = 2 * np.pi
    return (np.abs(_time_to_degree(t1) - _time_to_degree(t2))) * (circumference / 360)

_time_distance(time(6,44,22), time(7))

0.068213284932111679

In [32]:
ddf = DenseDepartureTimes(0.03)
train_ddt = ddf.fit_transform(train.copy())
bdte = BayesDepartureTimeEstimator()
bdte = bdte.fit(train_ddt)

counter_no_prob = 0

for i, row in test.iterrows():
    x = pd.DataFrame(data=[[row['start_lat'], row['start_lon']]], index=[i], columns=['lat', 'lon'])
    pred = bdte.predict_proba(x)
    sorted_pred = sorted(pred.items(), key=operator.itemgetter(1), reverse=True)
    for key in pred.keys():
        time_wa, time_wb = bdte.resolve_start_time_cluster(key)
        dummydate = date(1970, 1, 1)
        #time_wa = (datetime.combine(dummydate,time_wa)-timedelta(hours=1)).time()
        #time_wb = (datetime.combine(dummydate,time_wb)+timedelta(hours=1)).time()
        if len(x.between_time(time_wa, time_wb)) == 1:
            
            
            if str((row['start_cluster'], row['end_cluster'])) == key.split('_')[0]:
                print(x)
                print(key)
                print(time_wa, time_wb)
                print(row['start_cluster'], row['end_cluster'])
                for i, s_pred in enumerate(sorted_pred):
                    if s_pred[0] == key:
                        print('Ranked:', i+1, 'of total', len(pred), 'predictions')
                counter_no_prob += 1
        
    
print('Number of no prob:', counter_no_prob)
print('Total predictions:', len(test))

Number of no prob: 0
Total predictions: 20
